In [ ]:
R.version.string # checked difference between colab file and r installed version on pc.

[1] "R version 4.5.1 (2025-06-13)"

# packages

In [ ]:
install.packages("pracma") # for erf
install.packages("numDeriv")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



# Original Code for NEO DATA

In [ ]:
library(stats)
library(stats4)
library(pracma) # for erf
set.seed(15) # for reproducibility

# define data (neo dataset)
df <- read.csv("nearest-earth-objects(1910-2024).csv")
diameter_col <- df[, c(5)]
# based on five no summary and fences, upper inner bound for data to be within support.
dataset <- diameter_col[!is.na(diameter_col) & diameter_col<0.7126185 & diameter_col>0] # & diameter_col<1 & diameter_col>0.1
data <- sample(dataset, 200)
# NOTE: Results may be different due to version differences of sample method, so use .rds for version that was tested under (R Vers 4.4.2)

# NCBL Dist
#ncbl_pdf, used for modeling optim params
ncbl_pdf <- function(params, x) {
    lamb <- params[1]
    alpha <- params[2]
    mu <- params[3]
    sigma <- params[4]
    cLamb = (2 * atanh(1 - 2 * lamb)) / (1 - 2*lamb)
    f_r = (2 * atanh(1 - 2 * lamb) * (lamb ^ x) *((1 - lamb) ^ (1 - x))) / (1-(2 * lamb))
    F_r = ((lamb^x) * ((1 - lamb) ^ (1 - x)) + lamb - 1) / ((2 * lamb) - 1)
    s_r = 1 - F_r
    z = exp(-(alpha * log(F_r /(1 - F_r)) - mu)^2 /(2 * sigma^2))
    numerator = z * alpha * (s_r + F_r) * f_r
    denom = sqrt(2 * pi) * s_r * F_r * sigma
    f = numerator / denom
}

#ncbl_cdf, used for ks test later
ncbl_cdf <- function(params, x) {
    lamb <- params[1]
    alpha <- params[2]
    mu <- params[3]
    sigma <- params[4]
    f_r <- (2 * atanh(1 - 2 * lamb) * (lamb ^ x) *((1 - lamb) ^ (1 - x))) / (1-(2 * lamb))
    F_r <- ((lamb^x) * ((1 - lamb) ^ (1 - x)) + lamb - 1) / ((2 * lamb) - 1)
    s_r <- 1 - F_r
    f <- (1 / 2) * (1 + erf((alpha * log(F_r / s_r) - mu) / (sigma * sqrt(2))))
}

#define log likelihood function
ncbl_log_likelihood <- function(params, x) {
  lamb <- params[1]
  alpha <- params[2]
  mu <- params[3]
  sigma <- params[4]

  cLamb = (2 * atanh(1 - 2 * lamb)) / (1 - 2*lamb)
  f_r = (2 * atanh(1 - 2 * lamb) * (lamb ^ x) *((1 - lamb) ^ (1 - x))) / (1-(2 * lamb))
  F_r = ((lamb^x) * ((1 - lamb) ^ (1 - x)) + lamb - 1) / ((2 * lamb) - 1)
  s_r = 1 - F_r
  z = exp(-(alpha * log(F_r /(1 - F_r)) - mu)^2 /(2 * sigma^2))
  numerator = z * alpha * (s_r + F_r) * f_r
  denom = sqrt(2 * pi) * s_r * F_r * sigma
  f = numerator / denom

  ll = -sum(log(f))
}

# mle using l-bfgs-b algorithm
bfgs_optimization_ncbl <- function(data) {
  init_params <- c(0.1, 0.5, 0.2, 0.9)

  result <- optim(
    par = init_params,
    fn = ncbl_log_likelihood,
    x = data,
    method = "L-BFGS-B",
    lower = c(1e-10, 1e-10, -Inf, 1e-10), #lower bounds
    upper = c(1-(1e-10), Inf, Inf, Inf), #upper bounds
    control = list(maxit = 1000, trace = 3),
    hessian = TRUE
  )

  return(result)
}


# CB Dist
cb_pdf <- function(params, x) {
    lamb <- params[1]
    cLamb = (2 * atanh(1 - 2 * lamb)) / (1 - 2*lamb)
    f_r = (2 * atanh(1 - 2 * lamb) * (lamb ^ x) *((1 - lamb) ^ (1 - x))) / (1-(2 * lamb))
}

#cb cdf, used for ks test
cb_cdf <- function(params, x) {
  lamb <- params[1]
  F_r = ((lamb^x) * ((1 - lamb) ^ (1 - x)) + lamb - 1) / ((2 * lamb) - 1)
}

cb_log_likelihood <- function(params, x) {
    lamb <- params[1]
    x = data
    cLamb = (2 * atanh(1 - 2 * lamb)) / (1 - 2*lamb)
    f = (2 * atanh(1 - 2 * lamb) * (lamb ^ x) *((1 - lamb) ^ (1 - x))) / (1-(2 * lamb))
    l = prod(f)
    ll = -log(l)
}

# MLE
bfgs_optimization_cb <- function(params, x) {
  init_params <- c(0.001)

  result <- optim(
    par = init_params,
    fn = cb_log_likelihood,
    x = data,
    method = "Brent",
    lower = c(0.01), #lower bounds
    upper = c(0.99), #upper bounds
    control = list(maxit = 1000, trace = 3),
    hessian = TRUE
  )

  return(result)
}
# Expo Dist
exp_pdf <- function(params, x) {
  alpha <- params[1]
  f <- alpha * exp(-alpha * x)
}

exp_cdf <- function(params, x) {
  alpha <- params[1]
  f <- 1 - exp(-alpha * x)
}

exp_log_likelihood <- function(params, x) {
  alpha <- params[1]
  f <- alpha * exp(-alpha * x)
  l <- prod(f)
  ll <- -log(l)
}

bfgs_optimization_exp <- function(params, x) {
  init_params <- c(.1)

  result <- optim(
    par = init_params,
    fn = exp_log_likelihood,
    x = data,
    method = "L-BFGS-B",
    lower = c(0.1), #lower bounds 1e-6
    upper = c(Inf), #upper bounds
    control = list(maxit = 1000, trace = 3),
    hessian = TRUE
  )

  return(result)
}
# Beta Dist
beta_pdf <- function(params, x) {
  alpha <- params[1]
  beta <- params[2]
  dbeta(x, alpha, beta)
}

beta_pdf_v2 <- function(params, x) {
  alpha <- params[1]
  beta_par <- params[2]
  f <- (1 / beta(alpha, beta_par)) * x ^ (alpha-1) * (1-x) ^ (beta_par - 1)
}

beta_cdf <- function(params, x) {
  alpha <- params[1]
  beta <- params[2]
  pbeta(x, alpha, beta)
}

beta_log_likelihood <- function(params, x) {
  alpha <- params[1]
  beta_par <- params[2]
  f <- (1 / beta(alpha, beta_par)) * x ^ (alpha-1) * (1-x) ^ (beta_par - 1)
  l <- prod(f)
  ll <- -log(l)
}

bfgs_optimization_beta <- function(params, x) {
    init_params <- c(1.9, 4.5)

  result <- optim(
    par = init_params,
    fn = beta_log_likelihood,
    x = data,
    method = "L-BFGS-B",
    lower = c(1e-5,1e-5), #lower bounds 1e-5
    upper = c(Inf,Inf), #upper bounds
    control = list(maxit = 1000, trace = 3),
    hessian = TRUE
  )

  return(result)
}

##data section
##run lBFGSb optimization
optim_res_ncbl <- bfgs_optimization_ncbl(data) #ncbl
optim_res_cb <- bfgs_optimization_cb(data) #cb, couldnt get lbfgsb to work, so brent for method
optim_res_exp <- bfgs_optimization_exp(data) #exp
optim_res_beta <- bfgs_optimization_beta(data) #beta

print(optim_res_ncbl)
print(optim_res_cb)
print(optim_res_exp)
print(optim_res_beta)

#get params of optimized params (for modeling)
res_params_ncbl <- optim_res_ncbl$par
res_params_cb <- optim_res_cb$par
res_params_exp <- optim_res_exp$par
res_params_beta <- optim_res_beta$par

epsilon <- 1e-3 # define epsilon for covariance matrix making positive values

# covariance matrices section
hess_ncbl <- optim_res_ncbl$hessian
epsilon_ncbl <- max(abs(diag(hess_ncbl))) * epsilon  # Set epsilon based on the largest diagonal element
cov_ncbl <- solve(hess_ncbl + diag(epsilon_ncbl, nrow(hess_ncbl)))

cat("COVARIANCE MATRIX OF NCBL\n")
print(cov_ncbl)

cat("STANDARD ERRORS (LAMBDA, ALPHA, MU, SIGMA)\n")
print(diag(sqrt(cov_ncbl)))

hess_cb <- optim_res_cb$hessian
epsilon_cb <- max(abs(diag(hess_cb))) * epsilon  # Set epsilon based on the largest diagonal element
cov_cb <- solve(hess_cb + diag(epsilon_cb, nrow(hess_cb)))
cat("COVARIANCE MATRIX OF CB\n")
print(cov_cb)
cat("STANDARD ERRORS (LAMBDA)\n")
print(diag(sqrt(cov_cb)))

hess_exp <- optim_res_exp$hessian
epsilon_exp <- max(abs(diag(hess_exp))) * epsilon
cov_exp <- solve(hess_exp + diag(epsilon_exp, nrow(hess_exp)))
cat("COVARIANCE MATRIX OF EXP\n")
print(cov_exp)
cat("STANDARD ERRORS (ALPHA)\n")
print(diag(sqrt(cov_exp)))

hess_beta <- optim_res_beta$hessian
epsilon_beta <- max(abs(diag(hess_beta))) * epsilon
cov_beta <- solve(hess_beta + diag(epsilon_beta, nrow(hess_beta)))
cat("COVARIANCE MATRIX OF BETA\n")
print(cov_beta)
cat("STANDARD ERRORS (ALPHA, BETA)\n")
print(diag(sqrt(cov_beta)))
# tests section
# define empirical cdf for use in ks statistic
ecdf_data <- ecdf(data)
x_values <- seq(min(data), max(data), length.out = 1000)
ecdf_values <- ecdf_data(x_values)
n <- length(data)

# for computing p-value of ks test, emulates pkstwo()
pkstwo <- function(x) {
  if (x < 0) return(0)
  return(1 - (1 - x)^(2))
}

# ncbl tests (exp of -ll gives og l)
ncbl_likelihood_val <- exp(optim_res_ncbl$value)
cat("NCBL LIKELIHOOD: \n", ncbl_likelihood_val,"\n")
cat("NCBL AIC\n")

ncbl_aic <- 2 * 4 + 2 * optim_res_ncbl$value
print(ncbl_aic)

cat("NCBL BIC\n")
ncbl_bic <- log(length(data)) * 4 + 2 * optim_res_ncbl$value
print(ncbl_bic)

#ncbl ks test since ks.test won't work properly
ncbl_theoretical_values <- ncbl_cdf(res_params_ncbl, x_values)
ncbl_differences <- abs(ecdf_values - ncbl_theoretical_values)
ncbl_ks <- max(ncbl_differences)
ncbl_pval <- pkstwo(ncbl_ks)
cat("K-S STATISTIC", ncbl_ks, "\n", "P-VAL", ncbl_pval, "\n")

cb_likelihood_val <- exp(optim_res_cb$value)
cat("CB LIKE: ", cb_likelihood_val)
cat("CB AIC\n")
cb_aic <- 2 * 3 + 2 * optim_res_cb$value
print(cb_aic)
cat("CB BIC\n")
cb_bic <- log(length(data)) * 3 + 2 * optim_res_cb$value
print(cb_bic)

# # #ks test for cb
cb_theoretical_values <- cb_cdf(res_params_cb, x_values)
cb_differences <- abs(ecdf_values - cb_theoretical_values)
cb_ks <- max(cb_differences)
cb_pval <- pkstwo(cb_ks)

cat("K-S STATISTIC", cb_ks, "\n", "P-VAL", cb_pval, "\n")

# Exponential
exp_likelihood_val <- exp(optim_res_exp$value)
cat("EXP LIKE:", exp_likelihood_val)
cat("EXP AIC\n")
exp_aic <- 2 * 3 + 2 * optim_res_exp$value
print(exp_aic)
cat("EXP BIC\n")
exp_bic <- log(length(data)) * 3 + 2 * optim_res_exp$value
print(exp_bic)

exp_theoretical_values <- exp_cdf(res_params_exp, x_values)
exp_differences <- abs(ecdf_values - exp_theoretical_values)
exp_ks <- max(exp_differences)
exp_pval <- pkstwo(exp_ks)

cat("K-S STATISTIC", exp_ks, "\n", "P-VAL", exp_pval, "\n")

# Beta
beta_likelihood_val <- exp(optim_res_beta$value)
cat("BETA LIKE: \n", beta_likelihood_val, "\n")
cat("BETA AIC\n")
beta_aic <- 2 * 3 + 2 * optim_res_beta$value
print(beta_aic)
cat("BETA BIC\n")
beta_bic <- log(length(data)) * 3 + 2 * optim_res_beta$value
print(beta_bic)

#ks test for cb
beta_theoretical_values <- beta_cdf(res_params_beta, x_values)
beta_differences <- abs(ecdf_values - beta_theoretical_values)
beta_ks <- max(beta_differences)
beta_pval <- pkstwo(beta_ks)

cat("K-S STATISTIC", beta_ks, "\n", "P-VAL", beta_pval, "\n")

#modeling section
pdf("neo_plot.pdf", width = 10) # comment for non-landscape graph
# pdf("neo_plot.pdf") # uncomment for regular render
x1 <- seq(0, 1, by=1e-4) # 1e-4 for smoothing
hist(data, freq = FALSE, main = "Estimated Minimum Diameters",
    xlab = "Distance (in km)", ylab = "Density", xlim = c(0,1), ylim = c(0,4.75)) #, , xaxt = 'n'
lines(x1, ncbl_pdf(res_params_ncbl,x1), col = "#3C7A89", type = "l", lty = 4, lwd = 2)
lines(x1, beta_pdf(res_params_beta,x1), col = "#020122", type = "l", lty = 1, lwd = 2)
lines(x1, cb_pdf(res_params_cb, x1), col = "#FF934F", type = "l", lty = 5, lwd = 2)
lines(x1, exp_pdf(res_params_exp, x1), col = "#D30C7B", type = "l", lty = 6, lwd = 2)
legend(0.75,2, legend = c("NCBL","CB","EXP", "BETA"),
      col = c("#3C7A89","#020122", "#FF934F", "#D30C7B"), lty = c(4,5,6,1), lwd = 2)
dev.off()

N = 4, M = 5 machine precision = 2.22045e-16
At X0, 0 variables are exactly at the bounds
At iterate     0  f=      -69.178  |proj g|=       150.91
At iterate    10  f =      -123.75  |proj g|=        2.7169

iterations 17
function evaluations 18
segments explored during Cauchy searches 18
BFGS updates skipped 0
active bounds at final generalized Cauchy point 0
norm of the final projected gradient 0.00282906
final function value -123.825

F = -123.825
final  value -123.825397 
converged
N = 1, M = 5 machine precision = 2.22045e-16
At X0, 1 variables are exactly at the bounds
At iterate     0  f=       464.72  |proj g|=         1948
At iterate    10  f =      -111.91  |proj g|=    4.9246e-07

iterations 10
function evaluations 12
segments explored during Cauchy searches 10
BFGS updates skipped 0
active bounds at final generalized Cauchy point 0
norm of the final projected gradient 4.92456e-07
final function value -111.911

F = -111.911
final  value -111.910733 
converged
N = 2, M = 5 ma

Warning message in sqrt(cov_ncbl):
“NaNs produced”


[1] 0.3834245 0.4327573 0.7672140 0.5996429
COVARIANCE MATRIX OF CB
             [,1]
[1,] 1.459734e-05
STANDARD ERRORS (LAMBDA)
[1] 0.003820646
COVARIANCE MATRIX OF EXP
          [,1]
[1,] 0.1130178
STANDARD ERRORS (ALPHA)
[1] 0.3361813
COVARIANCE MATRIX OF BETA
            [,1]       [,2]
[1,] 0.005702674 0.01764955
[2,] 0.017649548 0.11031831
STANDARD ERRORS (ALPHA, BETA)
[1] 0.07551605 0.33214201
NCBL LIKELIHOOD: 
 1.672296e-54 
NCBL AIC
[1] -239.6508
NCBL BIC
[1] -226.4575
K-S STATISTIC 0.07082182 
 P-VAL 0.1366279 
CB LIKE:  3.614384e-50CB AIC
[1] -221.6887
CB BIC
[1] -211.7937
K-S STATISTIC 0.09418151 
 P-VAL 0.1794929 
EXP LIKE: 2.499114e-49EXP AIC
[1] -217.8215
EXP BIC
[1] -207.9265
K-S STATISTIC 0.08272902 
 P-VAL 0.1586139 
BETA LIKE: 
 7.273924e-50 
BETA AIC
[1] -220.2899
BETA BIC
[1] -210.395
K-S STATISTIC 0.1206897 
 P-VAL 0.2268134 


agg_record_1892164166 
                    2

# refactor with num derivs

In [20]:
library(stats)
library(stats4)
library(numDeriv)
library(pracma)

##### ---- ORIGINAL CODE FOR REFERENCE ---- #####
#set.seed(15) # seed of original testing
# load in data (original code)
#df <- read.csv("nearest-earth-objects(1910-2024).csv")
#diameter_col <- df[, c(5)]
# based on five no summary and fences, upper inner bound for data to be within support.
#dataset <- diameter_col[!is.na(diameter_col) & diameter_col<0.7126185 & diameter_col>0] # & diameter_col<1 & diameter_col>0.1
#data <- sample(dataset, 200)

# load rds files
data <- readRDS("sampled_neo_4_4_2.rds")

# x values for plotting and ks testing
x_values <- seq(min(data), max(data), length.out = 1000)
ecdf_data <- ecdf(data)
ecdf_vals <- ecdf_data(x_values)

# pkstwo hard coded because I couldn't get library to work
pkstwo <- function(x) {
  if (x < 0) return(0)
  return(1 - (1 - x)^2)
}

# list of distributions and components
distributions <- list(
  # ncbl list of elements
  ncbl = list(
    name = "NCBL",
    pdf = function(params, x) {
      lambda <- params[1]; alpha <- params[2]; mu <- params[3]; sigma <- params[4]
      f_r <- (2 * atanh(1 - 2 * lambda) * (lambda^x) * ((1 - lambda)^(1 - x))) / (1 - 2 * lambda)
      F_r <- ((lambda^x) * ((1 - lambda)^(1 - x)) + lambda - 1) / ((2 * lambda) - 1)
      s_r <- 1 - F_r
      z <- exp(-(alpha * log(F_r / s_r) - mu)^2 / (2 * sigma^2))
      num <- z * alpha * (s_r + F_r) * f_r
      denom <- sqrt(2 * pi) * s_r * F_r * sigma
      num / denom
    },
    cdf = function(params, x) {
      lambda <- params[1]; alpha <- params[2]; mu <- params[3]; sigma <- params[4]
      F_r <- ((lambda^x) * ((1 - lambda)^(1 - x)) + lambda - 1) / ((2 * lambda) - 1)
      s_r <- 1 - F_r
      0.5 * (1 + erf((alpha * log(F_r / s_r) - mu) / (sigma * sqrt(2))))
    },
    log_likelihood = function(params, x) {
      f <- distributions$ncbl$pdf(params, x)
      -sum(log(f))
    },
    init = c(0.1, 0.5, 0.2, 0.9),
    lower = c(1e-10, 1e-10, -Inf, 1e-10),
    upper = c(1 - 1e-10, Inf, Inf, Inf),
    method = "L-BFGS-B"
  ),
  # cb list of elements
  cb = list(
    name = "CB",
    pdf = function(params, x) {
      lambda <- params[1]
      (2 * atanh(1 - 2 * lambda) * (lambda^x) * ((1 - lambda)^(1 - x))) / (1 - 2 * lambda)
    },
    cdf = function(params, x) {
      lambda <- params[1]
      ((lambda^x) * ((1 - lambda)^(1 - x)) + lambda - 1) / ((2 * lambda) - 1)
    },
    log_likelihood = function(params, x) {
      f <- distributions$cb$pdf(params, x)
      -sum(log(f))
    },
    init = c(0.05),
    lower = c(0.01),
    upper = c(0.99),
    method = "Brent"
  ),
  exp = list(
    name = "EXP",
    pdf = function(params, x) {
      alpha <- params[1]
      alpha * exp(-alpha * x)
    },
    cdf = function(params, x) {
      alpha <- params[1]
      1 - exp(-alpha * x)
    },
    log_likelihood = function(params, x) {
      f <- distributions$exp$pdf(params, x)
      -sum(log(f))
    },
    init = c(0.1),
    lower = c(0.1),
    upper = c(Inf),
    method = "L-BFGS-B"
  ),
  # beta list of elements
  beta = list(
    name = "BETA",
    pdf = function(params, x) {
      alpha <- params[1]; beta <- params[2]
      dbeta(x, alpha, beta)
    },
    cdf = function(params, x) {
      alpha <- params[1]; beta <- params[2]
      pbeta(x, alpha, beta)
    },
    log_likelihood = function(params, x) {
      f <- distributions$beta$pdf(params, x)
      -sum(log(f))
    },
    init = c(1.9, 4.5),
    lower = c(1e-5, 1e-5),
    upper = c(Inf, Inf),
    method = "L-BFGS-B"
  )
)

# gradient and hessian helper functions
make_grad_fn <- function(ll_fn, data) {
  function(params) grad(function(p) ll_fn(p, data), params)
}

make_hess_fn <- function(ll_fn, data) {
  function(params) hessian(function(p) ll_fn(p, data), params)
}

# fit distribution using numerical gradient
fit_distribution <- function(dist, data) {
  grad_fn <- make_grad_fn(dist$log_likelihood, data)
  hess_fn <- make_hess_fn(dist$log_likelihood, data)

  result <- optim(
    par = dist$init,
    fn = function(p) dist$log_likelihood(p, data),
    gr = grad_fn,
    method = dist$method,
    lower = dist$lower,
    upper = dist$upper,
    control = list(maxit = 1000, trace = 3)
  )

  result$hessian <- hess_fn(result$par)
  return(result)
}

# results
evaluate_fit <- function(result, dist_name, data_len, num_params, cdf_func, data, x_vals, ecdf_vals) {
  loglik <- result$value
  aic <- 2 * num_params + 2 * loglik
  bic <- log(data_len) * num_params + 2 * loglik
  epsilon <- max(abs(diag(result$hessian))) * 1e-3
  cov_matrix <- solve(result$hessian + diag(epsilon, nrow(result$hessian)))
  se <- sqrt(diag(cov_matrix))

  theoretical_cdf <- cdf_func(result$par, x_vals)
  ks_stat <- max(abs(ecdf_vals - theoretical_cdf))
  pval <- pkstwo(ks_stat)

  cat(sprintf("=== %s ===\n", dist_name))
  cat("Estimated Parameters:\n"); print(result$par)
  cat("Final Value (NLL):", loglik, "\n")
  cat("Log-Likelihood:", exp(loglik), "\n")
  cat("AIC:", aic, "\n")
  cat("BIC:", bic, "\n")
  cat("K-S Statistic:", ks_stat, "\n")
  cat("P-Value:", pval, "\n")
  cat("Standard Errors:\n"); print(se)
  cat("\n")

  list(aic = aic, bic = bic, ks = ks_stat, pval = pval, se = se, cov = cov_matrix)
}

# testing stuff
results <- lapply(distributions, fit_distribution, data = data)
evaluations <- mapply(
  FUN = evaluate_fit,
  result = results,
  dist_name = names(distributions),
  num_params = sapply(distributions, function(d) length(d$init)),
  cdf_func = lapply(distributions, function(d) d$cdf),
  MoreArgs = list(data_len = length(data), data = data, x_vals = x_values, ecdf_vals = ecdf_vals),
  SIMPLIFY = FALSE
)

# plotting
pdf("neo_plot.pdf", width = 10)
hist(data, freq = FALSE, main = "Estimated Minimum Diameters",
     xlab = "Distance (in km)", ylab = "Density", xlim = c(0, 1), ylim = c(0, 4.75))
x1 <- seq(0, 1, by = 1e-4)
colors <- c("#000000", "#566E3D", "#FF934F", "#D30C7B")
lt <- c(1, 4, 5, 6)
i <- 1
for (name in names(distributions)) {
  lines(x1, distributions[[name]]$pdf(results[[name]]$par, x1), col = colors[i], lty = lt[i], lwd = 2)
  i <- i + 1
}
legend(0.75, 2, legend = toupper(names(distributions)), col = colors, lty = lt, lwd = 2)
dev.off()

N = 4, M = 5 machine precision = 2.22045e-16
At X0, 0 variables are exactly at the bounds
At iterate     0  f=      -69.178  |proj g|=       150.91
At iterate    10  f =      -123.75  |proj g|=        2.7203

iterations 17
function evaluations 18
segments explored during Cauchy searches 18
BFGS updates skipped 0
active bounds at final generalized Cauchy point 0
norm of the final projected gradient 0.00285954
final function value -123.825

F = -123.825
final  value -123.825397 
converged
N = 1, M = 5 machine precision = 2.22045e-16
At X0, 1 variables are exactly at the bounds
At iterate     0  f=       464.72  |proj g|=         1958
At iterate    10  f =      -111.91  |proj g|=    5.0221e-07

iterations 10
function evaluations 12
segments explored during Cauchy searches 10
BFGS updates skipped 0
active bounds at final generalized Cauchy point 0
norm of the final projected gradient 5.02212e-07
final function value -111.911

F = -111.911
final  value -111.910733 
converged
N = 2, M = 5 ma

agg_record_1881355274 
                    2

# Abalone Weights

In [28]:
library(stats)
library(stats4)
library(numDeriv)
library(pracma)

##### ---- ORIGINAL CODE FOR REFERENCE ---- #####
#set.seed(15)
#df2 <- read.csv("abalone_cleaned_col5_data_v2.csv")
#df2_main_col <- df2[, c(1)]
#data <- sample(df2_main_col, 100) # 100 samples of data

# load rds files
data <- readRDS("abalone_weights_sample_4-4-2.rds")

# x values for plotting and ks testing
x_values <- seq(min(data), max(data), length.out = 1000)
ecdf_data <- ecdf(data)
ecdf_vals <- ecdf_data(x_values)

# pkstwo hard coded because I couldn't get library to work
pkstwo <- function(x) {
  if (x < 0) return(0)
  return(1 - (1 - x)^2)
}

# list of distributions and components
distributions <- list(
  # ncbl list of elements
  ncbl = list(
    name = "NCBL",
    pdf = function(params, x) {
      lambda <- params[1]; alpha <- params[2]; mu <- params[3]; sigma <- params[4]
      f_r <- (2 * atanh(1 - 2 * lambda) * (lambda^x) * ((1 - lambda)^(1 - x))) / (1 - 2 * lambda)
      F_r <- ((lambda^x) * ((1 - lambda)^(1 - x)) + lambda - 1) / ((2 * lambda) - 1)
      s_r <- 1 - F_r
      z <- exp(-(alpha * log(F_r / s_r) - mu)^2 / (2 * sigma^2))
      num <- z * alpha * (s_r + F_r) * f_r
      denom <- sqrt(2 * pi) * s_r * F_r * sigma
      num / denom
    },
    cdf = function(params, x) {
      lambda <- params[1]; alpha <- params[2]; mu <- params[3]; sigma <- params[4]
      F_r <- ((lambda^x) * ((1 - lambda)^(1 - x)) + lambda - 1) / ((2 * lambda) - 1)
      s_r <- 1 - F_r
      0.5 * (1 + erf((alpha * log(F_r / s_r) - mu) / (sigma * sqrt(2))))
    },
    log_likelihood = function(params, x) {
      f <- distributions$ncbl$pdf(params, x)
      -sum(log(f))
    },
    init = c(0.01, 0.5, 0.2, 0.9),
    lower = c(1e-4, 1e-10, -Inf, 1e-10),
    upper = c(1 - 1e-4, Inf, Inf, Inf),
    method = "L-BFGS-B"
  ),
  # cb list of elements
  cb = list(
    name = "CB",
    pdf = function(params, x) {
      lambda <- params[1]
      (2 * atanh(1 - 2 * lambda) * (lambda^x) * ((1 - lambda)^(1 - x))) / (1 - 2 * lambda)
    },
    cdf = function(params, x) {
      lambda <- params[1]
      ((lambda^x) * ((1 - lambda)^(1 - x)) + lambda - 1) / ((2 * lambda) - 1)
    },
    log_likelihood = function(params, x) {
      f <- distributions$cb$pdf(params, x)
      -sum(log(f))
    },
    init = c(0.05),
    lower = c(0.01),
    upper = c(0.99),
    method = "Brent"
  ),
  # beta list of elements
  beta = list(
    name = "BETA",
    pdf = function(params, x) {
      alpha <- params[1]; beta <- params[2]
      dbeta(x, alpha, beta)
    },
    cdf = function(params, x) {
      alpha <- params[1]; beta <- params[2]
      pbeta(x, alpha, beta)
    },
    log_likelihood = function(params, x) {
      f <- distributions$beta$pdf(params, x)
      -sum(log(f))
    },
    init = c(1.9, 4.5),
    lower = c(1e-5, 1e-5),
    upper = c(Inf, Inf),
    method = "L-BFGS-B"
  )
)

# gradient and hessian helper functions
make_grad_fn <- function(ll_fn, data) {
  function(params) grad(function(p) ll_fn(p, data), params)
}

make_hess_fn <- function(ll_fn, data) {
  function(params) hessian(function(p) ll_fn(p, data), params)
}

# fit distribution using numerical gradient
fit_distribution <- function(dist, data) {
  grad_fn <- make_grad_fn(dist$log_likelihood, data)
  hess_fn <- make_hess_fn(dist$log_likelihood, data)

  result <- optim(
    par = dist$init,
    fn = function(p) dist$log_likelihood(p, data),
    gr = grad_fn,
    method = dist$method,
    lower = dist$lower,
    upper = dist$upper,
    control = list(maxit = 1000, trace = 3)
  )

  result$hessian <- hess_fn(result$par)
  return(result)
}

# results
evaluate_fit <- function(result, dist_name, data_len, num_params, cdf_func, data, x_vals, ecdf_vals) {
  loglik <- result$value
  aic <- 2 * num_params + 2 * loglik
  bic <- log(data_len) * num_params + 2 * loglik
  epsilon <- max(abs(diag(result$hessian))) * 1e-3
  cov_matrix <- solve(result$hessian + diag(epsilon, nrow(result$hessian)))
  se <- sqrt(diag(cov_matrix))

  theoretical_cdf <- cdf_func(result$par, x_vals)
  ks_stat <- max(abs(ecdf_vals - theoretical_cdf))
  pval <- pkstwo(ks_stat)

  cat(sprintf("=== %s ===\n", dist_name))
  cat("Estimated Parameters:\n"); print(result$par)
  cat("Final Value (NLL):", loglik, "\n")
  cat("Log-Likelihood:", exp(loglik), "\n")
  cat("AIC:", aic, "\n")
  cat("BIC:", bic, "\n")
  cat("K-S Statistic:", ks_stat, "\n")
  cat("P-Value:", pval, "\n")
  cat("Standard Errors:\n"); print(se)
  cat("\n")

  list(aic = aic, bic = bic, ks = ks_stat, pval = pval, se = se, cov = cov_matrix)
}

# testing stuff
results <- lapply(distributions, fit_distribution, data = data)
evaluations <- mapply(
  FUN = evaluate_fit,
  result = results,
  dist_name = names(distributions),
  num_params = sapply(distributions, function(d) length(d$init)),
  cdf_func = lapply(distributions, function(d) d$cdf),
  MoreArgs = list(data_len = length(data), data = data, x_vals = x_values, ecdf_vals = ecdf_vals),
  SIMPLIFY = FALSE
)

# plotting
pdf("abalone_weights.pdf", width = 10) # comment for non-landscape render
# pdf("abalone_weights.pdf") # uncomment for normal dimensions
hist(data, freq = FALSE, main = "Abalone Weights",
     xlab = "Weight (in grams)", ylab = "Density", xlim = c(0, 1), ylim = c(0, 2.75))
x1 <- seq(0, 1, by = 1e-4)
colors <- c("#000000", "#566E3D", "#D30C7B")
lt <- c(1, 4, 6)
i <- 1
for (name in names(distributions)) {
  lines(x1, distributions[[name]]$pdf(results[[name]]$par, x1), col = colors[i], lty = lt[i], lwd = 2)
  i <- i + 1
}
legend(0.75, 2, legend = toupper(names(distributions)), col = colors, lty = lt, lwd = 2)
dev.off()

N = 4, M = 5 machine precision = 2.22045e-16
At X0, 0 variables are exactly at the bounds
At iterate     0  f=      -38.185  |proj g|=       75.341
At iterate    10  f =      -45.274  |proj g|=        6.8799
At iterate    20  f =      -47.013  |proj g|=        3.5232
At iterate    30  f =      -47.083  |proj g|=     0.0004386

iterations 31
function evaluations 44
segments explored during Cauchy searches 33
BFGS updates skipped 0
active bounds at final generalized Cauchy point 0
norm of the final projected gradient 4.87903e-05
final function value -47.0826

F = -47.0826
final  value -47.082628 
converged
N = 2, M = 5 machine precision = 2.22045e-16
At X0, 0 variables are exactly at the bounds
At iterate     0  f=      -42.448  |proj g|=       4.7557

iterations 6
function evaluations 9
segments explored during Cauchy searches 7
BFGS updates skipped 0
active bounds at final generalized Cauchy point 0
norm of the final projected gradient 5.66461e-05
final function value -46.0359

F = -46

agg_record_1445755642 
                    2